## Import

In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary
import torch.nn.functional as F

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

/home/MMI24limbyungwan/.conda/envs/byungwan_resn/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
2


## Utils

In [2]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

In [3]:
import torch
import torch.nn as nn
import torch.autograd as autograd

class GradReverseFunction(autograd.Function):
    @staticmethod
    def forward(ctx, x, l):
        ctx.l = l
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.l, None

def grad_reverse(x, l):
    return GradReverseFunction.apply(x, l)

class GradReverse(nn.Module):
    def __init__(self, l):
        super(GradReverse, self).__init__()
        self.l = nn.Parameter(torch.tensor(l), requires_grad=False)

    def forward(self, x):
        return grad_reverse(x, self.l)


## Dataset, Data Loader

출력이미지 크기 키우기->ex) resnet 2048->1024->512->256 conv 256->512->1024->2048

In [4]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path)
        #mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

class CustomDataset_target(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image


        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            

        return image
     

transform = A.Compose(
    [   
        A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        # A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        # A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

Warmup

In [5]:
# class WarmUpLR(_LRScheduler):
#     def __init__(self, optimizer, total_iters, last_epoch=-1):
#         self.total_iters = total_iters
#         super(WarmUpLR, self).__init__(optimizer, last_epoch)

#     def get_lr(self):
#         return [base_lr * self.last_epoch / self.total_iters for base_lr in self.base_lrs]


## Define Model

In [6]:
# #Unet의 기본이 되는 conv블럭
# class ConvBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(ConvBlock, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.relu1 = nn.ReLU()
        
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
#         self.bn2 = nn.BatchNorm2d(out_channels)
#         self.relu2 = nn.ReLU()

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu1(x)
        
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu2(x)
#         return x

# #인코더 블럭
# class EncoderBlock(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(EncoderBlock,self).__init__()
#         self.convblock1 = ConvBlock(in_channels, out_channels)  # 첫 번째 ConvBlock의 in_channels는 입력 이미지의 채널 수와 일치해야 합니다.
#         #self.convblock2 = ConvBlock(out_channels, out_channels)  # 두 번째 ConvBlock의 in_channels는 out_channels와 일치해야 합니다.
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

#     def forward(self,x):
#         x = self.convblock1(x)
#         #x = self.convblock2(x)
#         p = self.maxpool(x)
#         return x , p
# #디코더 블럭
# #디코더는 업샘플링 이후 스킵연결과 붙어서 convblock을 통과해야함
# #skip보다 작은 x x먼저 업샘플링 32 -> 64 , skip과 결합 6464 
# class DecoderBlock(nn.Module):
#     def __init__(self, channels):
#         super(DecoderBlock,self).__init__()
#         self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1)#x 업샘플링
#         self.convblock1 = ConvBlock(channels*2, channels)#차원감소
#         #self.convblock2 = ConvBlock(channels, channels)
#     def forward(self,x,skip):
#         x = self.upsample(x)
#         x = torch.cat([x, skip], dim=1)
#         x = self.convblock1(x)
#         #x = self.convblock2(x)
#         return x
        
# #Unet구조 middle의 xm값의 움직임에 주의
# class Unet(nn.Module):
#     def __init__(self,n_classes):
#         super(Unet,self).__init__()
#         self.encoder1 = EncoderBlock(3,64)
#         self.encoder2 = EncoderBlock(64,128)
#         self.encoder3 = EncoderBlock(128,256)
#         self.encoder4 = EncoderBlock(256,512)
        
#         self.middleconv = ConvBlock(512,1024)
        
        
#         self.decoder4 = DecoderBlock(512)
#         self.decoder3 = DecoderBlock(256)
#         self.decoder2 = DecoderBlock(128)
#         self.decoder1 = DecoderBlock(64)
#         self.segmap = nn.Conv2d(64,n_classes, kernel_size=1)
        
#         ##
#         # self.domain_classifier = nn.Sequential(
#         #     nn.Conv3d(1024, 64, kernel_size=3, padding=1),
#         #     nn.ReLU(),
#         #     nn.Conv3d(64, 32, kernel_size=3, padding=1),
#         #     nn.ReLU(),
#         #     nn.Flatten(),
#         #     nn.Linear(32 * 16 * 16 * 16, 1),
#         #     nn.Sigmoid()
#         # )

#         # self.grad_reverse = GradReverse(0.0)
#         ##

#     def forward(self,x):
#         x1,p = self.encoder1(x)#3->64   #P:256,256 x1 :512,512
#         x2,p = self.encoder2(p)#64->128 #P:128,128 x2:256,256
#         x3,p = self.encoder3(p)#128->256#p:64,64 x3:128,128
#         x4,p = self.encoder4(p)#256->512#p:32,32 x4:64,64
        
#         xm = self.middleconv(p)#512->1024#32,32
        
#         x = self.decoder4(xm,x4)#뉴런:1024->512->512 #출력tensor:64,64
#         x = self.decoder3(x,x3)#뉴런:512->256->256 #출력tensor:128,128
#         x = self.decoder2(x,x2)#뉴런:256->128->128 #출력tensor:256,256
#         x = self.decoder1(x,x1)#뉴런:128->64->64 #출력tensor:512,512
#         x = self.segmap(x)

#         # 도메인 분류기에 GradReverse 적용
#         #feat = self.grad_reverse(x)
#         #domain_output = self.domain_classifier(x)

#         return x

## ResNet50

In [7]:
#Unet의 기본이 되는 conv블럭
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels,kernel_size = 3):
        super(ConvBlock, self).__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

class IdentityBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        
        # 1x1 convolution
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.relu1 = nn.ReLU()

        # 3x3 convolution
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.relu2 = nn.ReLU()

        # 1x1 convolution
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu3 = nn.ReLU()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
         
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        
        return out
class HeadBlock(IdentityBlock):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1):
        super(HeadBlock, self).__init__(in_channels, mid_channels, out_channels, stride)
        
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        identity = x
        out = super().forward(x)
        
        if identity.size() != out.size():
            identity = F.interpolate(identity, size=out.size()[2:])
        identity = self.shortcut(identity)
        
        out += identity
        out = self.relu3(out)
        
        return out

#인코더 블럭
class Conv2(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv2,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        return x , p
class Conv3(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv3,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock3 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        return x , p
class Conv4(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv4,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock3 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock4 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock5 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        x = self.identityblock5(x)
        p = self.maxpool(x)
        return x , p
class Conv5(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv5,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        return x , p
#디코더 블럭
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1, output_padding=1) # output_padding 추가
        self.convblock1 = ConvBlock(channels*2, channels)

    def forward(self, x, skip):
        x = self.upsample(x)
        if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
            x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        return x

#Unet구조 middle의 xm값의 움직임에 주의
class Unet(nn.Module):
    def __init__(self,n_classes):
        super(Unet,self).__init__()
        self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2)
        self.fbn1 = nn.BatchNorm2d(64)
        self.frelu1 = nn.ReLU()
        self.fconv2 = nn.Conv2d(64, 128, kernel_size=1, stride=1)
        self.fbn2 = nn.BatchNorm2d(128)
        self.frelu2 = nn.ReLU()
        self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.conv2 = Conv2(128,64,256)
        self.conv3 = Conv3(256,128,512)
        self.conv4 = Conv4(512,256,1024)
        self.conv5 = Conv5(1024,512,2048)
        
        self.middleconv = ConvBlock(2048,4096)
        #self.dropout = nn.Dropout2d(0.2) #
           
        self.decoder5 = DecoderBlock(2048)
        self.decoder4 = DecoderBlock(1024)
        self.decoder3 = DecoderBlock(512)
        self.decoder2 = DecoderBlock(256)
        self.decoder1 = DecoderBlock(128)
        
        self.segmap = nn.Conv2d(128,n_classes, kernel_size=1)
        
        
    def forward(self,x):
        x = self.fconv1(x)#3->64
        x = self.fbn1(x)
        x = self.frelu1(x)
        x = self.fconv2(x)#3->64
        x = self.fbn2(x)
        x1 = self.frelu2(x)
        p = self.fmaxpooling(x)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
        #print("첫 conv:",x0.shape,p.shape)
        x2,p = self.conv2(p)#conv2:  x1:([8, 256, 54, 54]) p([8, 256, 26, 26])
        #print("conv2: ",x1.shape, p.shape)
        x3,p = self.conv3(p)#conv3:  x2([8, 512, 26, 26]) p([8, 512, 12, 12])
        #print("conv3: ",x2.shape, p.shape)
        x4,p = self.conv4(p)#conv4:  x3([8, 1024, 12, 12]) p([8, 1024, 5, 5])
        #print("conv4: ",x3.shape, p.shape)
        x5,p = self.conv5(p)#conv5:  x4([8, 2048, 5, 5]) p([8, 2048, 2, 2])
        #print("conv5: ",x4.shape, p.shape)
        xm = self.middleconv(p)#xm([8, 4096, 2, 2])
        #xm = self.dropout(xm)
        #print("mid:",xm.shape)
        x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
        #print(x.shape)
        x = self.decoder4(x,x4)#뉴런:1024*2->1024 
        #print(x.shape)
        x = self.decoder3(x,x3) #14
        #print(x.shape)
        x = self.decoder2(x,x2)#28
        #print(x.shape)
        x = self.decoder1(x,x1)#55
        
        x = F.interpolate(x, size=(224, 224))
        x = self.segmap(x)
        #print(x.shape)
        return x

In [8]:
import pandas as pd
import os
N_CLASSES = 13
# 데이터셋을 불러옵니다.
csv_file = os.path.join("/mnt/nas27/Dataset/Samsung_DM", './train_source.csv')
data = pd.read_csv(csv_file)

# 클래스별 샘플 수를 저장할 리스트를 초기화합니다.
class_sample_counts = [0] * N_CLASSES  # num_classes는 클래스의 총 개수입니다.

# 데이터셋을 순회하면서 클래스별 샘플 수를 세고 저장합니다.
for idx in range(len(data)):
    mask_path = os.path.join("/mnt/nas27/Dataset/Samsung_DM", data.iloc[idx, 2][2:])  # 마스크 이미지 경로
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask[mask == 255] = 12  # 배경 클래스로 처리

    # 각 클래스의 샘플 수를 카운트합니다.
    for class_id in range(N_CLASSES):
        class_sample_counts[class_id] += (mask == class_id).sum()

# 클래스별 가중치를 계산합니다.
total_samples = sum(class_sample_counts)
class_weights = [total_samples / count for count in class_sample_counts]

# 클래스별 가중치를 텐서로 변환합니다.
class_weights = torch.tensor(class_weights).to(device)
class_weights = class_weights.float()

In [9]:
print(class_sample_counts)
print(class_weights)

[739351233, 56771156, 901883337, 69484874, 27027133, 9088387, 28767055, 660576317, 1049773711, 5071862, 2069341, 448558171, 602728911]
tensor([   6.2232,   81.0473,    5.1017,   66.2180,  170.2419,  506.2671,
         159.9452,    6.9654,    4.3830,  907.1918, 2223.4863,   10.2576,
           7.6339], device='cuda:0')


In [10]:
LR = 0.001
EP = 20
BATCH_SIZE = 4
ACCMULATION_STEP = 1 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
# WUP_ITERS = 10  # 웜업을 위한 반복 횟수
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Unet(n_classes = N_CLASSES).to(device)

# loss function과 optimizer 정의

#criterion =nn.CrossEntropyLoss()
#domain_criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
optimizer.zero_grad() 
#optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, nesterov=True)
# Warmup을 위한 스케줄러 설정
# scheduler_warmup = WarmUpLR(optimizer, WUP_ITERS)

dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
target_dataset = CustomDataset_target(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_target.csv'), transform=transform)
target_dataloader = DataLoader(target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./test.csv'), transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

In [11]:
# import matplotlib.pyplot as plt

# # 데이터로더에서 배치 하나를 가져옵니다.
# dataiter = iter(dataloader)
# images, masks = next(dataiter)

# # 이미지를 시각화합니다.
# for i in range(images.size(0)):
#     image = images[i].permute(1, 2, 0).numpy()  # 이미지를 CHW에서 HWC로 변환
#     mask = masks[i].numpy()

#     plt.figure(figsize=(8, 8))
#     plt.subplot(1, 2, 1)
#     plt.imshow(image)
#     plt.title("Image")

#     plt.subplot(1, 2, 2)
#     plt.imshow(mask, cmap='gray')
#     plt.title("Mask")

#     plt.show()


In [12]:
# for images, masks in tqdm(dataloader):
#     a = images.float().to(device)
#     b = masks.long().to(device)
#     break

# # PyTorch Tensor를 NumPy 배열로 변환
# a_numpy = a[3].cpu().numpy()  # 첫 번째 이미지만 선택하거나 필요한 이미지를 선택하세요.
# a_numpy = np.transpose(a_numpy, (1, 2, 0))
# b_numpy = b[3].cpu().numpy()
# b_numpy = b_numpy*12

# # 이미지를 저장
# cv2.imwrite('image_source.png', a_numpy)  # 이미지 저장
# cv2.imwrite('image_mask.png', b_numpy)   # 마스크 저장

In [13]:

# for epoch in range(2):  # 5 에폭 동안 학습합니다.
          
#     # 클래스별 IoU를 누적할 리스트 초기화
#     train_class_ious = []
    
#     # 학습
#     model.train()
#     epoch_loss = 0
    
#     for images, masks in tqdm(dataloader):
#         images = images.float().to(device)
#         masks = masks.long().to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         mask_spueeze = masks.squeeze(1)
#         loss = criterion(outputs, masks.squeeze(1))
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

## Model Train

In [14]:
import random
torch.cuda.empty_cache()
import wandb


wandb.init(
    # set the wandb project where this run will be logged
    project="practice_10_06",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LR,
    "architecture": "CNN",
    "dataset": "Samsung",
    "epochs": EP,
    }
)

for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()
        # if (epoch+1) % ACCMULATION_STEP == 0:
        #     optimizer.step()
        #     optimizer.zero_grad()
            # # Warmup 스케줄러 업데이트
            # if epoch < WUP_ITERS:
            #     scheduler_warmup.step()
        
        epoch_loss += loss.item()

        # train 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(N_CLASSES):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            train_class_ious.append(iou)

    train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    train_class_ious = np.mean(train_class_ious, axis=1)
    print("--IoU Scores Train--")
    for class_id, iou in enumerate(train_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print()

    # mIoU 계산
    train_mIoU = np.mean(train_class_ious)

    # validation
    val_loss = 0
    val_class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)

            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                val_class_ious.append(iou)

    val_class_ious = np.array(val_class_ious).reshape(N_CLASSES, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    print("--IoU Scores Valid--")
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 7 == 0:
            print("")

    # mIoU 계산
    val_mIoU = np.mean(val_class_ious)

    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(epoch_loss/len(dataloader))}, Train mIoU Score: {train_mIoU:.4f}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")

    # log metrics to wandb
    wandb.log({"train score": train_mIoU, "train loss": epoch_loss})
    wandb.log({"val score": val_mIoU, "val loss": val_loss})
    
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


100%|██████████| 549/549 [03:42<00:00,  2.46it/s]


--IoU Scores Train--
Class0: 0.2024 Class1: 0.2830 Class2: 0.3089 Class3: 0.3080 Class4: 0.3305 Class5: 0.3376 Class6: 0.3359 
Class7: 0.3411 Class8: 0.3276 Class9: 0.3408 Class10: 0.3446 Class11: 0.3489 Class12: 0.3640 

100%|██████████| 117/117 [00:16<00:00,  7.09it/s]


--IoU Scores Valid--
Class0: 0.2804 Class1: 0.2844 Class2: 0.2833 Class3: 0.2808 Class4: 0.2755 Class5: 0.2677 Class6: 0.2862 
Class7: 0.2766 Class8: 0.2697 Class9: 0.2820 Class10: 0.2819 Class11: 0.2619 Class12: 0.2751 
Epoch1
Train Loss: 1.1225906774428807, Train mIoU Score: 0.3210
Validation Loss: 1.2620958135678217, Validation mIoU Score: 0.2773
___________________________________________________________________________________________



...

--IoU Scores Train--
Class0: 0.5297 Class1: 0.5144 Class2: 0.5055 Class3: 0.5164 Class4: 0.5042 Class5: 0.5259 Class6: 0.5301 
Class7: 0.5191 Class8: 0.5186 Class9: 0.5166 Class10: 0.5096 Class11: 0.4891 Class12: 0.4441 

100%|██████████| 117/117 [00:16<00:00,  6.94it/s]

--IoU Scores Valid--
Class0: 0.3247 Class1: 0.3213 Class2: 0.3374 Class3: 0.3396 Class4: 0.3250 Class5: 0.3245 Class6: 0.3268 
Class7: 0.3240 Class8: 0.3150 Class9: 0.3310 Class10: 0.3213 Class11: 0.3057 Class12: 0.3189 
Epoch20
Train Loss: 0.3486968608393261, Train mIoU Score: 0.5095
Validation Loss: 1.3049740735282245, Validation mIoU Score: 0.3242
___________________________________________________________________________________________



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
train score,▁▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇███
val loss,▆▇▅▅▂▃▄█▅▄▃▄▁▂▅▁▂▁▂▆
val score,▁▂▅▄▆▇▅▄▅▅█▆▇█▆▇▇██▄
train loss,191.43458
train score,0.50949
val loss,152.68197
val score,0.32424


In [15]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(valid_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         for class_id in range(N_CLASSES):
#             iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
#             result.append(iou)

#     result = np.array(result).reshape(N_CLASSES, -1)
#     result = np.mean(result, axis=1)

In [16]:
# import random
# torch.cuda.empty_cache()

# for epoch in range(EP):
#     # 클래스별 IoU를 누적할 리스트 초기화
#     train_class_ious = []
#     # 학습
#     model.train()
#     epoch_loss = 0
#     for images in tqdm(target_dataloader):
#         images = images.float().to(device)

#         optimizer.zero_grad()
#         _, outputs = model(images)
#         target_domain = torch.zeros_like(outputs).to(device)
#         loss = domain_criterion(outputs, target_domain)
#         loss.backward()
#         optimizer.step()
        
#         epoch_loss += loss.item()

#         avg_domain_loss = epoch_loss / len(target_dataloader)
#     print(f"Epoch [{epoch + 1}/{EP}] - Domain Loss: {avg_domain_loss:.4f}")


In [17]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(valid_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         for class_id in range(N_CLASSES):
#             iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
#             result.append(iou)

#     result = np.array(result).reshape(N_CLASSES, -1)
#     result = np.mean(result, axis=1)

## Inference

In [18]:
# test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [19]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
        

## Submission

In [20]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result
# submit

In [21]:
# submit.to_csv('./baseline_submit.csv', index=False)